In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns

import sys
sys.path.append('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/src/')
import _config

sys.path.append('/home/unix/maxwshen/')
from mylib import util

import pb_rs2_combine as parent_script

Using data folder:
 /ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/data/


Note: This notebook provides stats for the rising star table (Figure 6C). No figures or results dir are made at the moment.

In [2]:
## Load design df

modelexp_nm = 'modelexp_intsimple_rs'

stats_df = pd.read_csv(parent_script.out_dir + f'{modelexp_nm}-aggstats.csv', index_col = 0)
eval_df = pd.read_csv(parent_script.out_dir + f'{modelexp_nm}-evals.csv', index_col = 0)

eval_df.columns

Index(['Genotype', 'Inferred fitness', 'Rising star, predicted', 'Fitness',
       'Rising star, observed', 'random_seed', 'dataset', 'Name',
       'datasource_x', 'Unnamed: 0.1', 'data_readlen', 'data_noise',
       'data_num_proposed_gts', 'data_proposal_type', 'data_num_groups',
       'data_risingstar_num', 'datasource_y'],
      dtype='object')

In [3]:
## Load ground truth data

fitness_df = pd.read_csv('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/_fitness_pt/fullgt_fitness.csv', index_col = 0)
fitness_df.head()

fq_df = pd.read_csv('/ahg/regevdata/projects/CRISPR-libraries/prj2/evolution/badran/out/pb_e3_interpolate_multi/badran_pacbio_pivot_1pct.csv')
time_cols = [col for col in fq_df.columns if 'hrs' in col]
fq_df = fq_df.rename(columns = {col: str(idx) for idx, col in enumerate(time_cols)})
fq_df = fq_df.set_index('Abbrev genotype')

fq_df.head()

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
Abbrev genotype,,,,,,,,,,,,,,,,,,,,,
VIWS.DNGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.010436,0.101351,0.307207,0.516204,0.690665,0.806228,0.840179,0.858280,0.859680,0.800000
VIW..DNGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.007699,0.067568,0.100481,0.082836,0.054376,0.031142,0.035129,0.038845,0.042116,0.042424
VIW...N.ERI.YC.KSKL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.211217,0.067568,0.007564,0.000469,0.000023,0.000000,0.000014,0.000186,0.002463,0.030303
VIWS..NGE.I.YC.KS.L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012949,0.135135,0.240628,0.237527,0.186697,0.128028,0.088182,0.059538,0.039415,0.024242
VIWS.DNGE.I.YC.KSKL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000012,0.000144,0.001679,0.018182


## new

In [4]:
crit = (eval_df['data_noise'] == 0.05) & (eval_df['data_readlen'] == 1)
eval_dfs = eval_df[crit]


In [5]:
def get_table_stats(query_gt):
    '''
        - Finds timepoint of peak frequency in ground truth fq matrix
        - Finds earliest timepoint where query_gt was identified as a rising star, or None
        - Calculates advance notice (from above two)
        - Finds ground truth full-length frequency at earliest timepoint
        - Calculates fitness ratio to highest fq. full length genotype at earliest timepoint
    '''


    crit = (eval_dfs['Genotype'] == query_gt) & (eval_dfs['Rising star, predicted'] == True)
    dfs = eval_dfs[crit]
    if len(dfs) == 0:
        print(f'{query_gt} not identified as a rising star')
        return None

    t0 = min(dfs['data_risingstar_num'])
    
    peak_t = int(np.argmax(fq_df.loc[query_gt]))
    peak_fq = max(fq_df.loc[query_gt])
    
    time_res = 6
    advance_notice = (peak_t - t0) * time_res

#     if advance_notice < 0:
#         print(f'{query_gt} not identified as a rising star')
#         return None
    
    highest_gt_at_t0 = np.argmax(fq_df[str(t0)])
    query_fq_at_t0 = fq_df.loc[query_gt, str(t0)]

    highest_gt_at_t0_fitness = fitness_df.loc[fitness_df['Genotype'] == highest_gt_at_t0, 'Fitness'].iloc[0]
    query_gt_fitness = fitness_df.loc[fitness_df['Genotype'] == query_gt, 'Fitness'].iloc[0]

    fitness_ratio = query_gt_fitness / highest_gt_at_t0_fitness
    
    stats_d = dict()
    stats_d['Genotype'] = query_gt
    stats_d['Earliest timepoint (h)'] = t0 * time_res
    stats_d['Peak timepoint (h)'] = peak_t * time_res
    stats_d['Advance notice (h)'] = advance_notice
    stats_d['Earliest frequency'] = query_fq_at_t0
    stats_d['Peak frequency'] = peak_fq
    stats_d['Fitness ratio'] = fitness_ratio

    sdf = pd.DataFrame(stats_d, index = [0])
    return sdf

# sdf = get_table_stats('VIWS.DNGE.I.YC.KS.L')
# display(sdf)

sdf = get_table_stats('V..................')
display(sdf)

/ahg/regevdata/users/maxwshen/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


,Genotype,Earliest timepoint (h),Peak timepoint (h),Advance notice (h),Earliest frequency,Peak frequency,Fitness ratio
0,V..................,84,102,18,0.125,0.290073,1.694505


In [6]:
crit = (eval_df['data_noise'] == 0.05) & (eval_df['data_readlen'] == 1)
eval_dfs = eval_df[crit]

query_gts = [
    'V..................',
    'V...........YC.....',
    'V..........PYC.....',
    'VI..........YC.....',
    'VIW...N.E.I.YC.KS.L',
    'VI.........PYCDK...',
    'V...G......PYCDKS..',
    'VIW...NGE.I.YC.KS.L',
    'VIW...N.ERI.YC.KSKL',
    'VIWS.DNGE.I.YC.KS.L',
]

mdf = pd.DataFrame()
for query_gt in query_gts:
    sdf = get_table_stats(query_gt)
    if sdf is None:
        continue
    mdf = mdf.append(sdf, ignore_index = True)
    
display(mdf)

VI.........PYCDK... not identified as a rising star


,Genotype,Earliest timepoint (h),Peak timepoint (h),Advance notice (h),Earliest frequency,Peak frequency,Fitness ratio
0,V..................,84,102,18,0.125000,0.290073,1.694505
1,V...........YC.....,108,168,60,0.117647,0.973529,3.256217
2,V..........PYC.....,240,282,42,0.139818,0.180441,1.119558
3,VI..........YC.....,210,282,72,0.108117,0.180441,1.079740
4,VIW...N.E.I.YC.KS.L,318,354,36,0.610425,0.815487,1.000000
5,V...G......PYCDKS..,390,390,0,0.289078,0.289078,1.000000
6,VIW...NGE.I.YC.KS.L,414,408,-6,0.366373,0.505929,1.000000
7,VIW...N.ERI.YC.KSKL,426,444,18,0.456582,0.596902,1.000000
8,VIWS.DNGE.I.YC.KS.L,492,522,30,0.516204,0.859680,1.000000


## other cond

In [7]:
crit = (eval_df['data_noise'] == 0) & (eval_df['data_readlen'] == 100)
eval_dfs = eval_df[crit]

query_gts = [
    'V..................',
    'V...........YC.....',
    'V..........PYC.....',
    'VI..........YC.....',
    'VIW...N.E.I.YC.KS.L',
    'VI.........PYCDK...',
    'V...G......PYCDKS..',
    'VIW...NGE.I.YC.KS.L',
    'VIW...N.ERI.YC.KSKL',
    'VIWS.DNGE.I.YC.KS.L',
]

mdf = pd.DataFrame()
for query_gt in query_gts:
    sdf = get_table_stats(query_gt)
    if sdf is None:
        continue
    mdf = mdf.append(sdf, ignore_index = True)
    
display(mdf)

VI.........PYCDK... not identified as a rising star


,Genotype,Earliest timepoint (h),Peak timepoint (h),Advance notice (h),Earliest frequency,Peak frequency,Fitness ratio
0,V..................,84,102,18,0.125000,0.290073,1.694505
1,V...........YC.....,108,168,60,0.117647,0.973529,3.256217
2,V..........PYC.....,240,282,42,0.139818,0.180441,1.119558
3,VI..........YC.....,210,282,72,0.108117,0.180441,1.079740
4,VIW...N.E.I.YC.KS.L,318,354,36,0.610425,0.815487,1.000000
5,V...G......PYCDKS..,384,390,6,0.262478,0.289078,1.000000
6,VIW...NGE.I.YC.KS.L,414,408,-6,0.366373,0.505929,1.000000
7,VIW...N.ERI.YC.KSKL,426,444,18,0.456582,0.596902,1.000000
8,VIWS.DNGE.I.YC.KS.L,492,522,30,0.516204,0.859680,1.000000


In [8]:
crit = (eval_df['data_noise'] == 0) & (eval_df['data_readlen'] == 1)
eval_dfs = eval_df[crit]

query_gts = [
    'V..................',
    'V...........YC.....',
    'V..........PYC.....',
    'VI..........YC.....',
    'VIW...N.E.I.YC.KS.L',
    'VI.........PYCDK...',
    'V...G......PYCDKS..',
    'VIW...NGE.I.YC.KS.L',
    'VIW...N.ERI.YC.KSKL',
    'VIWS.DNGE.I.YC.KS.L',
]

mdf = pd.DataFrame()
for query_gt in query_gts:
    sdf = get_table_stats(query_gt)
    if sdf is None:
        continue
    mdf = mdf.append(sdf, ignore_index = True)
    
display(mdf)

VI.........PYCDK... not identified as a rising star


,Genotype,Earliest timepoint (h),Peak timepoint (h),Advance notice (h),Earliest frequency,Peak frequency,Fitness ratio
0,V..................,84,102,18,0.125000,0.290073,1.694505
1,V...........YC.....,108,168,60,0.117647,0.973529,3.256217
2,V..........PYC.....,240,282,42,0.139818,0.180441,1.119558
3,VI..........YC.....,210,282,72,0.108117,0.180441,1.079740
4,VIW...N.E.I.YC.KS.L,318,354,36,0.610425,0.815487,1.000000
5,V...G......PYCDKS..,390,390,0,0.289078,0.289078,1.000000
6,VIW...NGE.I.YC.KS.L,414,408,-6,0.366373,0.505929,1.000000
7,VIW...N.ERI.YC.KSKL,426,444,18,0.456582,0.596902,1.000000
8,VIWS.DNGE.I.YC.KS.L,492,522,30,0.516204,0.859680,1.000000


## investigate

In [9]:
crit = (eval_df['data_noise'] == 0) & (eval_df['data_readlen'] == 100)
eval_dfs = eval_df[crit]

eval_dfs[eval_dfs['data_risingstar_num'] == 57]

,Genotype,Inferred fitness,"Rising star, predicted",Fitness,"Rising star, observed",random_seed,dataset,Name,datasource_x,Unnamed: 0.1,data_readlen,data_noise,data_num_proposed_gts,data_proposal_type,data_num_groups,data_risingstar_num,datasource_y
8461,............YC.....,0.513858,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8462,VI....N.E....C....L,0.622296,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8463,VIW...N...I.YCDKS.L,1.335650,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8464,VIW.........YC.KS..,2.076653,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8465,VIW.......I.YC.KS..,1.914777,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8466,VIW...N.E...YC.KS.L,1.440554,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8467,V..S......I.YCD....,0.594783,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8468,VI...........C.....,0.553115,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8469,VI...........C.K...,0.608862,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate
8470,VI.................,0.475824,False,NaN,False,0,intsimple_smart_rl_100_noise_0_star_57,312,pb_interpolate,312,100,0.0,37,smart,12,57,pb_interpolate


## stats

In [10]:
stats_df.columns

Index(['random_seed', 'dataset', 'Name', 'datasource_x', 'Unnamed: 0.1',
       'data_readlen', 'data_noise', 'data_num_proposed_gts',
       'data_proposal_type', 'data_num_groups', 'data_risingstar_num',
       'datasource_y', 'Num pred rising stars', 'Num true rising stars',
       'True positive', 'False positive', 'True negative', 'False negative',
       'AUROC', 'Average precision score', 'Sensitivity', 'Specificity',
       'Precision', 'Recall'],
      dtype='object')

In [11]:
crit = (stats_df['data_noise'] == 0.05) & (stats_df['data_readlen'] == 1)
stats_dfs = stats_df[crit]

stats_dfs[['Sensitivity', 'Specificity', 'Precision', 'Recall']].describe()

,Sensitivity,Specificity,Precision,Recall
count,59.000000,87.000000,60.000000,59.000000
mean,0.496570,0.918093,0.525198,0.496570
std,0.414767,0.214574,0.424121,0.414767
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.955259,0.000000,0.000000
50%,0.500000,1.000000,0.500000,0.500000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [12]:
crit = (stats_df['data_noise'] == 0) & (stats_df['data_readlen'] == 100)
stats_dfs = stats_df[crit]

stats_dfs[['Sensitivity', 'Specificity', 'Precision', 'Recall']].describe()

,Sensitivity,Specificity,Precision,Recall
count,59.000000,88.000000,59.000000,59.000000
mean,0.383253,0.925542,0.423729,0.383253
std,0.415887,0.183020,0.436672,0.415887
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.963214,0.000000,0.000000
50%,0.285714,1.000000,0.333333,0.285714
75%,0.833333,1.000000,1.000000,0.833333
max,1.000000,1.000000,1.000000,1.000000
